# 作業目標: 使用python正規表達式對資料進行清洗處理

這份作業我們會使用詐欺郵件的文本資料來作為清洗與處理的操作。
[資料集](https://www.kaggle.com/rtatman/fraudulent-email-corpus/data#)

### 讀入資料文本
因原始文本較大，先使用部份擷取的**sample_emails.txt**來進行練習

In [1]:
#讀取文本資料
with open("./sample_emails.txt", "r") as f:
    sample_corpus = f.read()

In [2]:
sample_corpus[:1000]

'From r  Wed Oct 30 21:41:56 2002\nReturn-Path: <james_ngola2002@maktoob.com>\nX-Sieve: cmu-sieve 2.0\nReturn-Path: <james_ngola2002@maktoob.com>\nMessage-Id: <200210310241.g9V2fNm6028281@cs.CU>\nFrom: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>\nReply-To: james_ngola2002@maktoob.com\nTo: webmaster@aclweb.org\nDate: Thu, 31 Oct 2002 02:38:20 +0000\nSubject: URGENT BUSINESS ASSISTANCE AND PARTNERSHIP\nX-Mailer: Microsoft Outlook Express 5.00.2919.6900 DM\nMIME-Version: 1.0\nContent-Type: text/plain; charset="us-ascii"\nContent-Transfer-Encoding: 8bit\nX-MIME-Autoconverted: from quoted-printable to 8bit by sideshowmel.si.UM id g9V2foW24311\nStatus: O\n\nFROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-27-587908.\nE-MAIL: (james_ngola2002@maktoob.com).\n\nURGENT BUSINESS ASSISTANCE AND PARTNERSHIP.\n\n\nDEAR FRIEND,\n\nI AM ( DR.) JAMES NGOLA, THE PERSONAL ASSISTANCE TO THE LATE CONGOLESE (PRESIDENT LAURENT KABILA) WHO WAS ASSASSINATED BY HIS BODY GUARD ON 16TH JAN. 2001.\n\n\nTHE INCIDE

### 讀取寄件者資訊
觀察文本資料可以發現, 寄件者資訊都符合以下格式

`From: <收件者姓名> <收件者電子郵件>`

In [3]:
#<your code>#
import re
match = re.findall(pattern="From:.*.com\>", string=sample_corpus)
match

['From: "MR. JAMES NGOLA." <james_ngola2002@maktoob.com>',
 'From: "Mr. Ben Suleman" <bensul2004nng@spinfinder.com>',
 'From: "PRINCE OBONG ELEME" <obong_715@epatra.com>']

### 只讀取寄件者姓名

In [4]:
for line in match:
    search_obj = re.search(pattern="\".+\"", string=line)
    if search_obj: print(search_obj.group())

"MR. JAMES NGOLA."
"Mr. Ben Suleman"
"PRINCE OBONG ELEME"


### 只讀取寄件者電子信箱

In [5]:
emails = []
for line in match:
    search_obj = re.search(pattern="\w+@.+\.com", string=line)
    if search_obj: 
        email = search_obj.group()
        emails.append(email)
        print(email)

james_ngola2002@maktoob.com
bensul2004nng@spinfinder.com
obong_715@epatra.com


### 只讀取電子信箱中的寄件機構資訊
ex: james_ngola2002@maktoob.com --> 取maktoob

In [6]:
for email in emails:
    search_obj = re.search(pattern="(?<=@)\w+(?=\.com)", string=email)
    if search_obj:
        print(search_obj.group())

maktoob
spinfinder
epatra


### 結合上面的配對方式, 將寄件者的帳號與機構訊返回
ex: james_ngola2002@maktoob.com --> [james_ngola2002, maktoob]

In [7]:
for email in emails:
    findall_obj = re.findall(pattern=".+(?=\.com)", string=email)
    if len(findall_obj):
        ta, tb = re.split("@", findall_obj[0])
        print(f"{ta}, {tb}")

james_ngola2002, maktoob
bensul2004nng, spinfinder
obong_715, epatra


### 使用正規表達式對email資料進行處理
這裡我們會使用到python其他的套件協助處理(ex: pandas, email, etc)，這裡我們只需要專注在正規表達式上即可，其他的套件是方便我們整理與處理資料。

### 讀取與切分Email
讀入的email為一個長字串，利用正規表達式切割讀入的資料成一封一封的email，並將結果以list表示。

輸出: [email_1, email_2, email_3, ....]

In [8]:
###讀取文本資料:fradulent_emails.txt###
with open('all_emails.txt', 'r', encoding="utf-8", errors='ignore') as f:
    corpus = f.read()
    
###切割讀入的資料成一封一封的email###
emails = re.split(pattern="From r", string=corpus)

###我們可以使用list來儲存每一封email###
###注意！這裡請仔細觀察sample資料，看資料是如何切分不同email###
#<your code>#
emails = emails[1:]
len(emails) #查看有多少封email

3977

### 從文本中擷取所有寄件者與收件者的姓名和地址

In [9]:
import re
import pandas as pd
import email

emails_list = [] #創建空list來儲存所有email資訊

for mail in emails[:20]: #只取前20筆資料 (處理速度比較快)
    emails_dict = dict() #創建空字典儲存資訊
    ###取的寄件者姓名與地址###
    
    #Step1: 取的寄件者資訊 (hint: From:)
    search_obj_sender = re.search("(?<=From\:).*\<.*\>", mail)
    
    #Step2: 取的姓名與地址 (hint: 要注意有時會有沒取到配對的情況)
    if search_obj_sender:
        search_obj_name = re.search("(?<=\").+(?=\")", search_obj_sender.group())
        search_obj_mail = re.search("(?<=\<).+(?=\>)", search_obj_sender.group())
        
        #Step3: 將取得的姓名與地址存入字典中
        if search_obj_name and search_obj_mail:
            emails_dict['sender_name'] = search_obj_name.group()
            emails_dict['sender_mail'] = search_obj_mail.group()
        else:
            emails_dict['sender_name'] = None
            emails_dict['sender_mail'] = None

    ###取的收件者姓名與地址###
    
    #Step1: 取的寄件者資訊 (hint: To:)
    search_obj_recipient = re.search("(?<=\nTo:).+", mail)
    
    #Step2: 取的姓名與地址 (hint: 要注意有時會有沒取到配對的情況)
    #Step3: 將取得的姓名與地址存入字典中
    if search_obj_recipient:
        emails_dict['recipient_name'] = search_obj_recipient.group().strip().split("@")[0]
        emails_dict['recipient_mail'] = search_obj_recipient.group().strip()
    else:
        emails_dict['recipient_name'] = None
        emails_dict['recipient_mail'] = None
        
    ###取得信件日期###
    #Step1: 取得日期資訊 (hint: To:)
    search_obj_date = re.search("(?<=Date).*", mail)

    #Step2: 取得詳細日期(只需取得DD MMM YYYY)
    #Step3: 將取得的日期資訊存入字典中
    if search_obj_date:
        data_info = search_obj_date.group()
        emails_dict['date'] = re.search("\d+\s\w+\s\d+", data_info).group()
    else:
        emails_dict['date'] = None
        
    ###取得信件主旨###
    #Step1: 取得主旨資訊 (hint: Subject:)
    search_obj_subject = re.search("(?<=Subject:\s).*", mail)
    
    #Step2: 移除不必要文字 (hint: Subject: )
    #Step3: 將取得的主旨存入字典中
    if search_obj_subject:
        emails_dict['subject']= search_obj_subject.group()
    else:
        emails_dict['subject']= None
    
    ###取得信件內文###
    #這裡我們使用email package來取出email內文 (可以不需深究，本章節重點在正規表達式)
    try:
        full_email = email.message_from_string(mail)
        body = full_email.get_payload()
        emails_dict["email_body"] = body
    except:
        emails_dict["email_body"] = None
    
    ###將字典加入list###
    emails_list.append(emails_dict)

In [10]:
#將處理結果轉化為dataframe
emails_df = pd.DataFrame(emails_list)
emails_df

,sender_name,sender_mail,recipient_name,recipient_mail,date,subject,email_body
0,MR. JAMES NGOLA.,james_ngola2002@maktoob.com,webmaster,webmaster@aclweb.org,31 Oct 2002,URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...
1,Mr. Ben Suleman,bensul2004nng@spinfinder.com,R,R@M,31 Oct 2002,URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ..."
2,PRINCE OBONG ELEME,obong_715@epatra.com,webmaster,webmaster@aclweb.org,31 Oct 2002,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
3,PRINCE OBONG ELEME,obong_715@epatra.com,webmaster,webmaster@aclweb.org,31 Oct 2002,GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...
4,Maryam Abacha,m_abacha03@www.com,R,R@M,1 Nov 2002,I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope..."
5,None,None,davidkuta,davidkuta@yahoo.com,02 Nov 2002,Partnership,ATTENTION: ...
6,Barrister tunde dosumu,tunde_dosumu@lycos.com,None,None,None,Urgent Attention,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
7,William Drallo,william2244drallo@maktoob.com,webmaster,webmaster@aclweb.org,3 Nov 2002,URGENT BUSINESS PRPOSAL,FROM: WILLIAM DRALLO.\nCONFIDENTIAL TEL: 233-2...
8,MR USMAN ABDUL,abdul_817@rediffmail.com,R,R@M,04 Nov 2002,THANK YOU,"CHALLENGE SECURITIES LTD.\nLAGOS, NIGERIA\n\n\..."
9,Tunde Dosumu,barrister_td@lycos.com,None,None,None,Urgent Assistance,"Dear Sir,\n\nI am Barrister Tunde Dosumu (SAN)..."
